In [ ]:
# this version of the epoch function has the player id in 
# the format of a string with p_ prepended to it

def epoch(matches, players_list, players_dict):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.
    
    Future iterations of this function should do away with the p_ , and only should need the 
    players_dict with the keys as integers indicating the player id.
    '''
    
    players = ['p_' + str(p) for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # print(f'{players_compete=}')
    # update attributes for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = ['p_' + str(winner) for winner in matches[matches['winner_id']==int(player[2:])]['loser_id']]
        losses = ['p_' + str(winner) for winner in matches[matches['loser_id']==int(player[2:])]['winner_id']]
        # get opponents' rating and rd
        # print(f'{wins=}')
        losses_rating = [players_dict[loss].getRating() for loss in losses ]
        losses_rd = [players_dict[loss].getRd() for loss in losses ]
        wins_rating = [players_dict[win].getRating() for win in wins ]
        wins_rd = [players_dict[win].getRd() for win in wins ]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
        
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)

    return (players_list,players_dict)
    # opponent_indices = matches[matches['loser_id'].str==player '']